# パッケージインストール

In [ ]:
!pip install -U tensorflow
!pip install -U onnx
!pip install -U nvidia-pyindex
!pip install -U onnx-graphsurgeon
!pip install -U onnxsim
!pip install -U onnxruntime
!pip install -U simple_onnx_processing_tools
!pip install -U onnx2tf

# FLW-Netモデルロード

In [ ]:
!git clone https://github.com/hitzhangyu/FLW-Net

In [ ]:
%cd FLW-Net

In [ ]:
import torch
import model

scale_factor = 20
nbins = 14

FLW_net = model.enhance_net_nopool(scale_factor, nbins).cuda()
FLW_net.load_state_dict(torch.load('best_Epoch.pth'))

# ONNX変換

In [5]:
def convert_to_onnx(net, file_name='output.onnx', input_shape=(512, 512), device='cpu'):
    input_image = torch.randn(1, 3, input_shape[1], input_shape[0]).to(device)
    histogram = torch.randn(1, 15, 1, 1).to(device)

    input_layer_names = ['input_image', 'histogram']
    output_layer_names = ['enhanced_image', 'retouch_image', 'ill_image']

    torch.onnx.export(
        net, 
        (input_image, histogram),
        file_name, 
        verbose=True,
        opset_version=13,
    )

In [ ]:
convert_to_onnx(FLW_net, file_name='FLW-Net_320x240.onnx', input_shape=(320, 240), device='cuda:0')
convert_to_onnx(FLW_net, file_name='FLW-Net_640x480.onnx', input_shape=(640, 480), device='cuda:0')

In [ ]:
!onnxsim 'FLW-Net_320x240.onnx' 'FLW-Net_320x240.onnx'
!onnxsim 'FLW-Net_640x480.onnx' 'FLW-Net_640x480.onnx'

# onnx2tfを用いてTFLiteへ変換

In [ ]:
!onnx2tf -i FLW-Net_320x240.onnx -o saved_model_320x240

In [ ]:
!onnx2tf -i FLW-Net_640x480.onnx -o saved_model_640x480